In [17]:
import pandas as pd
import shutil
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.1.0


In [18]:
#train = pd.read_csv("C:/Users/dampe/Desktop/Maruti/Analytics/trainDataset.csv")
#print(train)

In [19]:
"""
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])
  
  """

"\ndef preprocess(df):\n  # clean up data we don't want to train on\n  # in other words, users will have to tell us the mother's age\n  # otherwise, our ML service won't work.\n  # these were chosen because they are such good predictors\n  # and because these are easy enough to collect\n  df = df[df.weight_pounds > 0]\n  df = df[df.mother_age > 0]\n  df = df[df.gestation_weeks > 0]\n  df = df[df.plurality > 0]\n  \n  # modify plurality field to be a string\n  twins_etc = dict(zip([1,2,3,4,5],\n                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))\n  df['plurality'].replace(twins_etc, inplace=True)\n  \n  # now create extra rows to simulate lack of ultrasound\n  nous = df.copy(deep=True)\n  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'\n  nous['is_male'] = 'Unknown'\n  \n  return pd.concat([df, nous])\n  \n  "

In [20]:
CSV_COLUMNS = 'Age,Job,Marital,Education,Default,Balance,HHInsurance,CarLoan,Communication,NoOfContacts,DaysPassed,PrevAttempts,Outcome,CarInsurance,Duration'.split(',')
LABEL_COLUMN = 'CarLoan'

DEFAULTS = [[0.0], ['null'], ['null'],['null'], ['0.0'], ['0.0'], ['0.0'], [0.0],['null'], [0.0],[0.0], [0], ['null'], [0], [0] ]
TRAIN_STEPS = 1000

In [21]:
def read_dataset(filename, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.compat.v1.decode_csv(value_column, record_defaults=DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
      return features, label
    
    # Create list of files that match pattern
    file_list = tf.compat.v1.gfile.Glob(filename)

    # Create dataset from file list
    dataset = (tf.compat.v1.data.TextLineDataset(file_list)  # Read text file
                 .map(decode_csv))  # Transform each elem by applying decode_csv fn
      
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size=10*batch_size)
    else:
        num_epochs = 1 # end-of-input after this
 
    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset
  return _input_fn

In [22]:
# Define feature columns
def get_categorical(name, values):
  return tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_vocabulary_list(name, values))

def get_cols():
  # Define column types
  return [\
          get_categorical('Job', ['management', 'admin.', 'blue-collar', 'services', 'technician', 'self-employed', 'entrepreneur', 'retired', 'NA', 'housemaid', 'student', 'unemployed']),
          tf.feature_column.numeric_column('Age'),
          tf.feature_column.numeric_column('Default'),
          get_categorical('Marital', ['divorced', 'married', 'single' ]),
          get_categorical('Education', ['NA', 'secondary', 'primary', 'tertiary' ]),
          tf.feature_column.numeric_column('Balance'),
          tf.feature_column.numeric_column('HHInsurance'),
          tf.feature_column.numeric_column('NoOfContacts'),
          get_categorical('Communication', ['cellular', 'NA', 'telephone' ]),
          tf.feature_column.numeric_column('DaysPassed'),
          tf.feature_column.numeric_column('PrevAttempts'),
          tf.feature_column.numeric_column('Duration'),
          get_categorical('Outcome', ['failure', 'NA', 'other', 'success' ])          
      ]

In [23]:
def serving_input_fn():
    feature_placeholders = {
        'Job': tf.compat.v1.placeholder(tf.string, [None]),
        'Age': tf.compat.v1.placeholder(tf.float32, [None]),
        'Default': tf.compat.v1.placeholder(tf.float32, [None]),
        'Balance': tf.compat.v1.placeholder(tf.float32, [None]),
        'Marital': tf.compat.v1.placeholder(tf.string, [None]),
        'Education': tf.compat.v1.placeholder(tf.string, [None]),
        'HHInsurance': tf.compat.v1.placeholder(tf.float32, [None]),
        'NoOfContacts': tf.compat.v1.placeholder(tf.float32, [None]),
        'Communication': tf.compat.v1.placeholder(tf.string, [None]),
        'NoOfContacts': tf.compat.v1.placeholder(tf.float32, [None]),
        'DaysPassed': tf.compat.v1.placeholder(tf.float32, [None]),
        'PrevAttempts': tf.compat.v1.placeholder(tf.float32, [None]),
        'Outcome': tf.compat.v1.placeholder(tf.string, [None]),
        'Duration': tf.compat.v1.placeholder(tf.float32, [None]),
    }
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [24]:
def train_and_evaluate(output_dir):
  EVAL_INTERVAL = 300
  run_config = tf.estimator.RunConfig(save_checkpoints_secs = EVAL_INTERVAL,
                                      keep_checkpoint_max = 3)
  estimator = tf.estimator.DNNRegressor(
                       model_dir = output_dir,
                       feature_columns = get_cols(),
                       hidden_units = [64, 32],
                       config = run_config)
  train_spec = tf.estimator.TrainSpec(
                       input_fn = read_dataset('C:/Users/dampe/Desktop/Maruti/Analytics/train2.csv', mode = tf.estimator.ModeKeys.TRAIN),
                       max_steps = TRAIN_STEPS)
  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  eval_spec = tf.estimator.EvalSpec(
                       input_fn = read_dataset('C:/Users/dampe/Desktop/Maruti/Analytics/eval.csv', mode = tf.estimator.ModeKeys.EVAL),
                       steps = None,
                       start_delay_secs = 60, # start evaluating after N seconds
                       throttle_secs = EVAL_INTERVAL,  # evaluate every N seconds
                       exporters = exporter)
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [25]:
# Run the model
shutil.rmtree('maruti_output', ignore_errors = True) # start fresh each time
tf.compat.v1.summary.FileWriterCache.clear()
train_and_evaluate('maruti_output')

INFO:tensorflow:Using config: {'_model_dir': 'maruti_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and e

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version